# Part I: Loading, exploring and cleaning data

![](img/cleaning.jpeg)


Data has been obtained from the Internet movie database:
- https://www.imdb.com/search/title?title_type=feature&release_date=1970-01-01,&countries=in&languages=hi&page=1

In this section we'll learn:
- How to load data from different sources
- How to combine data from different sources
- Exploring the content of the loaded data: the raw data, and statistics and aggregates
- Cleaning the data, removing wrong values, formatting it in a useful representation, dealing with missing values...

In [ ]:
import os

MOVIES_LIST_FNAME = os.path.join('data', 'movies_from_list.csv.gz')
MOVIES_DETAIL_FNAME = os.path.join('data', 'movies_from_detail.json.bz2')

### Load data

pandas supports many formats for loading data. You can see most of them in:
- https://pandas.pydata.org/pandas-docs/stable/api.html#input-output

Also, data can be imported into pandas from Python objects (list, dict, tuple...) with the `DataFrame` and `Series` constructor.

**EXERCISE:** Load the datasets defined in `MOVIES_LIST_FNAME` and `MOVIES_DETAIL_FNAME` into pandas `DataFrame` objects.

### Exploring

There are many ways to explore the data in a pandas container. Try calling on them the next methods: `.head()`, `.tail()`, `.head().T`, `.describe()` and `.info()`.

### Combining datasets

pandas provides different ways to combine datasets into one. For people familiar with SQL, the functionality is similar to `JOIN` and `UNION`.

Documentation can be found here:
- https://pandas.pydata.org/pandas-docs/stable/merging.html

In the case of adding columns of one `DataFrame` to another, we can use `DataFrame.join` (equivalent to SQL `JOIN`). Documentation can be found here:
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html

**EXERCISE:** In this case we want to join the two previously loaded `DataFrame` objects into one. To make sure we implement it correctly, we need to see the check the shape of both `DataFrame` with the method `.shape`, and also check the shape of the resulting `DataFrame` to make sure we are not duplicating or missing rows. When joining, the important parameters are `on` and `how`.

### Selecting data

pandas has two main containers, `Series` and `DataFrame`. You can think as a `DataFrame` as the content of a spreadsheet, or a SQL table. And `Series` as one of its columns.

More formally, a `DataFrame` is a 2-dimensional structure, where each element is labelled both by its column and its row.

| Movie | Year | Genre |
| --- | --- | --- |
| **PK** | 2014 | Comedy |
| **Kabhi Khushi Kabhie Gham...** | 2001 | Drama |
| **3 idiots** | 2009 | Comedy |

As you can see in the example, every value can be defined by its position based on its labels. For example, the value at `PK`/`Genre` is `Comedy`. In pandas, this would be `movies.loc['PK', 'Genre']`.

Also, every value cab be defined by its position. For example, in the second row, and first column, the value is `2001`. In pandas this would be `movies.iloc[1, 0]` (remember that in Python, the first index is `0`, the second is `1`...)

**Exercise:** Can you obtain the value in the column `storyline`, and the row `tt2905838` in the movies `DataFrame`. Can you guess which is the movie? :)

To select a whole column from a `DataFrame` (which will be a `Series`) we can use `movies['rating']` (like getting a value from a `dict`). If the name of the column does not contain special characters, we can also use `movies.rating` (like for a class attribute or a `namedtuple`).

Then, we can perform operations with the resulting `Series`:
- Compute the maximum, the minimum, the median, or any other provided statistic:
   - `movies['rating'].min()`
   - `movies['rating'].max()`
   - `movies['rating'].median()`
- Double the rating of each movie (if we feel generous): `movies['rating'] * 2`
- Make comparisons: `movies['rating'] > .8` (this will return a `Series` with `True` and `False` values, depending on whether the condition is satisfied in each row)

The last point is quite important, as boolean `Series` can be used to filter the data. The syntax is `movies[condition]` where condition is a `Series` or boolean values. Note that the syntax is the same as `movies[column_name]`, and pandas will check what is the type and size of what is between the squared brackets, to decide whether to return a column, or a filtered `DataFrame`.

**Exercise:** Return all the movies that have a rating greater than `9.5` and that their their title starts by the letter `K` (most Python string functions and methods can be applied to a `Series` by using `Series.str.<func-name>`).

### Checking data

In the dataset, we can see that both original `DataFrame` objects had a column for the user rating of the movies, `rate` and `rating`. This information may be redundant, but is worth comparing that they have the same values.

**Exercise:** Compare whether there are differences in the two columns, and keep a single column with the information that makes more sense. To delete columns, `DataFrame` provides a method `.drop()`, or it is also possible to use `del movies['rating']`

### Extracting information

There is some information that needs to be transformed before we can use it. For example, the column with the number of votes. If you take a look, you can see how instead of having the number of votes as a number, the column contains a string including thousand separators (e.g. `1,000`). This is how it was presented in the original source.

But this format is not appropiate to do operations. For example, if we want to check the smallest (`.min()`) or the largest (`.max()`) number of votes in our data, we're not getting the right results. This is because when dealing with strings, `'9'` is greater than `'10'` in the same was `z` is "greater" than `ab`.

**Exercises:**
1. Create a new column `rating_votes` with the number of votes converted as a number. The first step is to get a string representation that does not contain the commas. This can be done with the method `.str.replace()`, replacing the commas with empty strings. Second is to convert the column type, from a Python object to a float representation. This can be done with the method `.astype()`.

1. Get the smallest and largest values for the number of votes.

1. Check how many missing values we have in the data (pandas provides the methods `.isnull()` and `.notnull()`). Remember that in Python, `True == 1` and `False == 0`, so you can sum `True` and `False` values.

Another column that we need to preprocess before we use it in a meaningful way is the `release_date`. In this case, the best way to do it is probably with regular expressions. Regular expressions define a pattern that is used to extract data that matches it. In pandas, it's possible to extract data based on regular expressions with the method `.str.extract()`.

Some basics of regular expressions:
- To mark the beginning of the string use `^`, and `$` for the end.
- A digit (0 to 9) is matched by `\d`
- To match specific characters a range can be defined, for example `[a-z]` would match all lower case letters. They can be combined, like in `[a-zA-Z]`.
- To match several patterns of the same type, `*` can be used if the pattern is optional, and `+` if at least one character needs to be matched. For example `a+` would match `a`, `aa`, `aaa`...
- To match the exact number of characters, curly brackets can be used, for example `a{4}` would match `aaaa`.

**Exercise:** Write a regular expression that extracts the date from the `release_date` column. A string containing a date can be converted to a date type in pandas with `pandas.to_datetime()`

Finally, the last column that we want to preprocess is the duration of the movie. To operate with this column, once more we need it to be a numeric column, and not a string.

**Exercise:** Replace the column `runtime` by a numerical column with the number of minutes extracted from the original column.

### Save data

Finally, in order to persist the data that we have created (by transforming the original data), we can export it. There are many formats supported by `pandas`, they are listed here:
- https://pandas.pydata.org/pandas-docs/stable/api.html#id12

The type of format used will depend on the case. Here there are some examples:
- csv: When we want to open the data with a spreadsheet (discourage in other cases, as it looses type information, and it's inefficient)
- json: When there are complex data structures we want to preserve (like `list`, `dict`...)
- hdf and parquet: When we want to save the data in an efficient way (consume less storage and load faster)

**Exercise:** Save the data as JSON. Try saving just couple of rows first, with different values of `orient`, to see which one looks more appropriate. Also test how the parameter `lines` affects the output.